## Proposed features for weights and biases
Draft Rank = 35% <br>
Projected Season Points = 15% <br>
Positions needing to be filled on the team = 25% <br>
Position Value = 10% <br>
Drafter's Pro Team Preferences = 7% <br>
Player Injury Implications = 6% <br>
Star Power = 4% <br>

In [2]:
import json
import ast
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [3]:
# pull in our data from the csv
data = pd.read_csv('analysis/data/draft-players-2023.csv')

In [4]:
# The first column is the original indexes which are not needed, so we will drop it
data = data.drop(data.columns[0], axis=1)
data.head(5)

,draftRank,eligibleSlots,playerId,name,projPoints
0,1,"[11, 12, 13, 14, 16, 17]",39832,Shohei Ohtani,588.0
1,2,"[10, 5, 12, 16, 17]",36969,Juan Soto,602.0
2,3,"[13, 14, 16, 17]",32081,Gerrit Cole,629.0
3,4,"[3, 7, 19, 11, 12, 16, 17]",32801,Jose Ramirez,676.0
4,5,"[13, 14, 16, 17]",39878,Corbin Burnes,613.0


In [5]:
all_projected_points = data['projPoints'].sum()
print("The total projected points ESPN has calculated for all players in the 2023 season is:", f'{int(all_projected_points):,}')

The total projected points ESPN has calculated for all players in the 2023 season is: 150,474


## Position analysis
For each position that will be used in our league, we are going to evaluate the strength and value of players in those positions.
The positions our league uses is: Catcher, 1B, 2B, 3B, SS, 3 OF, DH, 4 Utility, and 9 Pitchers
SP are limited to 8 starts per week, so it is best to plan to have 6 SP and 3 RP

We are going to aggregate the following statistics:
- **Position Weight** = Projected fantasy score / Total fantasy point projections
- **Normalized Position Weight** = Position weight on a normalized scale
- **Max Projected Points** = Maximum amount of projected fantasy points for a player in this position
- **Min Projected Points** = The amount of fantasy points the lowest scoring drafted player would be projected to earn.  This is calculated as n = (# of starters for position * # of teams in league).  Ascending sorted projected fantasy players\[n]



In [6]:
# create an object of each position and their ID using the positions lookup file
position_map = {}
with open('lookups/espn-positions-map.json') as f:
  position_map = json.load(f)

In [7]:
# We will manually configure which IDs are being used by our league for positions
# and how many starters we want to have at each position to filter our positions object
STARTER_POSITION_IDS = [0, 1, 2, 3, 4, 5, 11, 14, 15]
POSITIONS_STARTERS_CONFIG = {"0": 1, "1": 1, "2": 1, "3": 1, "4": 1, "5": 3, "11": 1, "14": 6, "15": 3, "12": 4, "16": 5}

position_map = {k: v for k, v in position_map.items() if v['posId'] in STARTER_POSITION_IDS}
# Note: we just removed UTIL and BENCH from the position map, even though they exist in the starters_config dict
# This is appropriate for our up front calculations, but we will still consider this data later

# appending the starters allowed to each position
for k, v in position_map.items():
  position_map[k]['startersAllowed'] = POSITIONS_STARTERS_CONFIG[str(v['posId'])]

In [8]:
# We are going to do some analysis on each positon and update it to the position_map object
for position in STARTER_POSITION_IDS:

  # subset all player data based on who is eligible for the position sorted by projected points
  position_eligible_players = [values for index, values in data.iterrows() if position in ast.literal_eval(values['eligibleSlots'])]
  position_eligible_players = sorted(position_eligible_players, key=lambda k: k['projPoints'], reverse=True)

  # parse the max projected points, min projected points, and total projected points for the position
  max_projected_points = position_eligible_players[0]['projPoints']
  min_projected_points = position_eligible_players[(position_map[str(position)]['startersAllowed']*12)+1]['projPoints']
  sum_projected_points = sum([float(player['projPoints']) for player in position_eligible_players])

  # save each data point to the position_map object
  position_map[str(position)]['totalProjectedPoints'] = sum_projected_points
  position_map[str(position)]['maxProjectedPoints'] = max_projected_points
  position_map[str(position)]['minProjectedPoints'] = min_projected_points

In [9]:
# Let's take a look at an updated entry from the position_map object
position_map['0']

{'posId': 0,
 'name': 'Catcher',
 'abbrev': 'C',
 'startersAllowed': 1,
 'totalProjectedPoints': 10139.5,
 'maxProjectedPoints': 452.5,
 'minProjectedPoints': 316.5}

In [10]:
# We are going to calculate how many points each position is projected to contribute
# to overall scoring this year and turn that into a weighted score
for k, v in position_map.items():
  v['overallWeight'] = v['totalProjectedPoints'] / all_projected_points

In [11]:
# In order to do accurate analysis, we are also going to observe the normalized weight for each position
pos_points_list = [v['totalProjectedPoints'] for k, v in position_map.items()]
norm_pos_points_list = preprocessing.normalize([pos_points_list])[0]
position_map = [{**v, **{'normalizedPositionValue': norm_pos_points_list[i]}} for i, (k, v) in enumerate(position_map.items())]

In [12]:
print("Total Points:", pos_points_list)
print("Normalized Points:", norm_pos_points_list)
position_map[0]

Total Points: [10139.5, 15608.0, 15170.5, 16141.0, 13826.5, 35857.5, 22173.5, 43800.0, 18474.5]
Normalized Points: [0.14246494 0.21930005 0.21315296 0.22678896 0.1942691  0.50381544
 0.31154854 0.61541146 0.25957578]


{'posId': 0,
 'name': 'Catcher',
 'abbrev': 'C',
 'startersAllowed': 1,
 'totalProjectedPoints': 10139.5,
 'maxProjectedPoints': 452.5,
 'minProjectedPoints': 316.5,
 'overallWeight': 0.0673837340670149,
 'normalizedPositionValue': 0.14246494211383717}

## Evaluate Players:
We are going to weight the following features for individual players:
- What is the rank of the player compared to all projected points in their position
- Handle multiple positions through an averaging algorithm
- What is the normalized |weight of a players projected fantasy points
- How much of a negative impact to other teams does drafting the player create?

In [15]:
# loop through each player in our dataset and calculate their positional value
for index, player in data.iterrows():
  # identify the relevant starter positions for this player
  players_positions = [position for position in ast.literal_eval(player['eligibleSlots']) if position in STARTER_POSITION_IDS]

  # if there are multiple eligible positions, this will average them together
  scores = []
  for pos in players_positions:
    position_stats = [position for position in position_map if position['posId'] == pos]
    score = position_stats[0]['normalizedPositionValue'] * player['projPoints']
    scores.append(score)
  
  # calculate the position-adjusted score for the player based on their eligible positions
  players_weights = [position['overallWeight'] for position in position_map if position['posId'] in players_positions]
  normalized_players_weights = preprocessing.normalize([players_weights])[0]

  avg_score = np.mean(scores) * (len(players_positions) * sum(normalized_players_weights))

  # update the player's positional value in our original dataset
  data.loc[index, 'positionAdjustedPointsScore'] = avg_score

data.head(25)

,draftRank,eligibleSlots,playerId,name,projPoints,positionAdjustedScore
0,1,"[11, 12, 13, 14, 16, 17]",39832,Shohei Ohtani,588.0,732.470167
1,2,"[10, 5, 12, 16, 17]",36969,Juan Soto,602.0,303.296895
2,3,"[13, 14, 16, 17]",32081,Gerrit Cole,629.0,387.093806
3,4,"[3, 7, 19, 11, 12, 16, 17]",32801,Jose Ramirez,676.0,508.392321
4,5,"[13, 14, 16, 17]",39878,Corbin Burnes,613.0,377.247223
5,6,"[1, 7, 19, 11, 12, 16, 17]",35002,Vladimir Guerrero Jr.,595.0,440.090799
6,7,"[1, 7, 19, 12, 16, 17]",30193,Freddie Freeman,607.0,133.115128
7,8,"[9, 10, 5, 11, 12, 16, 17]",33192,Aaron Judge,612.0,686.858738
8,9,"[13, 14, 16, 17]",28976,Max Scherzer,510.0,313.859843
9,10,"[10, 5, 12, 16, 17]",33039,Mookie Betts,559.5,281.884739


In [24]:
preprocessing.normalize(list(data['positionAdjustedScore']))

ValueError: Expected 2D array, got 1D array instead:
array=[732.4701671  303.29689535 387.09380623 508.39232112 377.24722292
 440.09079929 133.11512793 686.85873799 313.85984289 281.88473912
 285.15953948 345.86123863 324.93724911 134.826035   293.55126482
 110.92765513 267.08857218 659.9231012  323.09101474 111.37241987
 116.90970739 332.32218658 321.24478037 431.95466687 104.41964033
 328.62971784 576.87155445 127.11521046 545.6183828  276.09086155
  96.74601095 130.37387735 324.32183765 421.22976504 327.39889493
 103.25402574 119.40438592 304.01325958 268.0298145  104.71104398
 298.47455647  90.82080345 306.47490541 102.52657217 295.39749919
 303.37964191 311.39819706 106.16649207 120.42493623 269.54126082
 546.00780397 315.70607726 293.55126482 300.32079084 272.26908916
 233.5184568  291.70503045 362.79754127 306.47490541 100.19624924
 291.08961899 231.25128732 345.78562802 549.93591767 273.24268675
 235.26456102  88.25576358 267.70398364  88.97524702 302.70196716
 237.29707261 267

## Determine Position Needs
- We will need to consider what fielding positions we still need on our team
- This should consider what positions we've already picked
- This should consider how much value is still left on the draft board for a given position at this moment in time
- This should consider how much value will still be left on the board by time we get to the next pick
* A super feature would be to predict, out of the most probable remaining draft pick outcomes from the current pick on, by all teams, which player at what pick delivers me the most value

In [16]:
# First, we are going to define the draft order for this season
draft_order_list = []

# We will use the draft-order.json file which describes the order that Fantasy teams will pick in the draft
with open('analysis/data/draft-order-2023.json', 'r') as f:
  draft_order = json.load(f)
  draft_order = sorted(draft_order, key=lambda k: k['draftOrder'])
  print("The draft order is:", [player['teamName'] for player in draft_order])
  rounds = 27

  # We will loop through the draft order and add the team name to the list for each pick
  # There will be some conditions applied to account for the "snake" draft order pattern
  for i in range(rounds):
    reverse_toggle = i % 2

    if i < 3:
      reverse_toggle = 1

    if reverse_toggle == 1:
      for player in draft_order:
        draft_order_list.append(player['teamName'])
    elif reverse_toggle == 0:
      for player in draft_order[::-1]:
        draft_order_list.append(player['teamName'])

print("There will be", len(draft_order_list), "picks over", int(len(draft_order_list) / len(draft_order)), "rounds")

The draft order is: ['BG 420', 'The Dissenters', "Loser's Club L", 'RIP Paulie Walnuts', 'Nomar Losing', 'Pecan Sandies', 'Rocket City Trash Pandas', '2-Time Champ Booch', 'Commissioner Getbot420', 'Big Trains', 'Hot Italian Snausages', 'Vandelay Industries']
There will be 324 picks over 27 rounds


In [17]:
# Hold all the owner info in an array
owners = []
with open ('analysis/data/owners.json') as f:
  owners = json.load(f)

# Update the owners object to include the positions they need to fill
for owner in owners:
  owner['posNeeds'] = {"0": 1, "1": 1, "2": 1, "3": 1, "4": 1, "5": 1, "11": 1, "14": 1, "15": 1}

Formula for calculating position needs:
$$
  \int \frac{StartersNeeded - \frac{1}{PlayersEligiblePositions}}{RosterSize -(PickNumber - 1)}
$$

In [18]:
def updatePositionNeeds(picker: str, owner_list: list, pick: dict):
  """
  This function will generate a pick for the given picker
  Parameters:
    picker (str): The name of the team that is picking
    owner_list (list): The list of owners with their current positional needs
    pick (dict): Details about the player that was just drafted
  """

  pos_scores = [owner['posNeeds'] for owner in owner_list if owner['teamName'] == picker]

  matching_positions = [position for position in ast.literal_eval(pick['eligibleSlots']) if int(position) in STARTER_POSITION_IDS]
  positions_player_can_fill = sum([v for k, v in POSITIONS_STARTERS_CONFIG.items() if int(k) in matching_positions]) + POSITIONS_STARTERS_CONFIG['16']
  for pos in matching_positions:
    players_eligible_positions = len(matching_positions)
    
    if pos in [0, 1, 2, 3, 4, 5, 11, 12]:
      starters_needed = positions_player_can_fill + POSITIONS_STARTERS_CONFIG['12']
    else:
      starters_needed = positions_player_can_fill
    
    teams_pick_number = int(np.ceil(pick['pickNumber'] / 12))
    result = (starters_needed - (1 / players_eligible_positions)) / (27 - (teams_pick_number - 1))
    new_score = pos_scores[0][str(pos)] * result
    # print("Old score:", pos_scores[0][str(pos)], "\nNew score:", new_score)
    owner_list['teamName' == picker]['posNeeds'][str(pos)] = new_score

In [19]:
for owner in owners:
  owner['picks'] = []

In [ ]:
pick_number = 1
drafted_players = []
for pick in draft_order_list:
  picker = pick
  remaining_players = data[~data['playerId'].isin(drafted_players)]
  drafted_player = generatePick(picker, pick_number, owners, drafted_players, remaining_players)
  updatePositionNeeds(picker, owners, drafted_player)
  owners['teamName' == picker]['picks'].append(drafted_player)
  drafted_players.append(drafted_player)
  pick_number += 1

In [ ]:
def generatePick(picker: str, pick_number: int, owner_list: list, current_pick_list: list, remaining_picks: pd.DataFrame):
  """
  This function will generate a pick for the given picker
  """
  PICK_ROUND = np.ceil(pick_number / 12)
  BATTER_ELIGIBLE_POSITIONS = [0, 1, 2, 3, 4, 5, 11, 12]
  PITCHER_ELIGIBLE_POSITIONS = [14, 15]

  draft_rank_score = 

  for index, player in remaining_picks.iterrows():
    remaining_picks.loc[index, 'draftRankScore'] = 0
    remaining_picks.loc[index, 'projectedSeasonPointsScore'] = 0
    remaining_picks.loc[index, 'draftersPositionalNeedsScore'] = 0

  for index, player in remaining_picks.iterrows():
    normalized_draft_rank_score = preprocessing.normalize([remaining_picks['draftRankScore']])[index]

  payload = {
    'pickNumber': pick_number,
    'pickRound': PICK_ROUND,
    'teamName': "2-Time Champ Booch",
    'teamId': 8,
    'playerId': '39832',
    'playerName': 'Shohei Ohtani',
    'draftRank': 1,
    'eligibleSlots': '[11, 12, 13, 14, 16, 17]',
    'projPoints': 1000,
  }


  return

In [ ]:
pick_number = 1
for picker in draft_order_list:
  for team in predicted_draft_picks:
    if team['teamName'] == picker:
      pick_index = generatePick(picker, pick_number, predicted_draft_picks, posEligData, data)
      team['picks'].append(posEligData.iloc[pick_index][['playerId', 'name']].to_dict())
      posEligData = posEligData.drop(pick_index)
      pick_number += 1
      team['picks'] = sorted(team['picks'], key=lambda k: k['playerId'])


In [ ]:
formula = sum({k: v for k, v in position_starters_config.items() if k in batter_pos}.values()) - sum({pick['name'] for pick in current_pick_list}.values())